In [50]:
# # The function name, type hints, and docstring are all part of the tool
# # schema that's passed to the model. Defining good, descriptive schemas
# # is an extension of prompt engineering and is an important part of
# # getting models to perform well.
# def add(a: int, b: int) -> int:
#     """Add two integers.

#     Args:
#         a: First integer
#         b: Second integer
#     """
#     return a + b


# def multiply(a: int, b: int) -> int:
#     """Multiply two integers.

#     Args:
#         a: First integer
#         b: Second integer
#     """
#     return a * b


# def greating(query: str) -> str:
#     """Welcome the user prompt by discriping the serivce that you provide.

#     Args:
#         query: the user prompt
#     """
#     return "This Sensorconnect framework"

In [61]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
import getpass
import os

# os.environ["GROQ_API_KEY"] = getpass.getpass()


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

@tool
def greating(query: str) -> str:
    """Welcome the user prompt by discriping the serivce that you provide.

    Args:
        query: the user prompt
    """
    return "This Sensorconnect framework"


tools = [add, multiply,greating]

In [52]:
# tools = [add, multiply, greating]

In [62]:


llm = ChatGroq(model="llama3-8b-8192")
llm_with_tools = llm.bind_tools(tools)

# query = "Hello"
query="what is 2 + 2"
AIMessage=llm_with_tools.invoke(query)

In [54]:
AIMessage

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t74c', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 1034, 'total_tokens': 1106, 'completion_time': 0.05798152, 'prompt_time': 0.379338634, 'queue_time': None, 'total_time': 0.437320154}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f9a41f4d-1f3c-4b73-a9ca-7faae56f4825-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 2}, 'id': 'call_t74c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1034, 'output_tokens': 72, 'total_tokens': 1106})

In [63]:
from langchain_core.messages import HumanMessage

query = "Hello"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'greating', 'args': {'query': 'Hello'}, 'id': 'call_9vpf', 'type': 'tool_call'}]


In [65]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply, "greating":greating}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='Hello'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9vpf', 'function': {'arguments': '{"query":"Hello"}', 'name': 'greating'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 1145, 'total_tokens': 1218, 'completion_time': 0.057639295, 'prompt_time': 0.173468779, 'queue_time': None, 'total_time': 0.231108074}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ed033eb3-e2b5-4c49-a81b-c8cda9bcbec0-0', tool_calls=[{'name': 'greating', 'args': {'query': 'Hello'}, 'id': 'call_9vpf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1145, 'output_tokens': 73, 'total_tokens': 1218}),
 ToolMessage(content='This Sensorconnect framework', name='greating', tool_call_id='call_9vpf')]

In [67]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_s3wr', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_5hjy', 'type': 'tool_call'}]


In [68]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply, "greating":greating}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_s3wr', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_5hjy', 'function': {'arguments': '{"a":11,"b":49}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 163, 'prompt_tokens': 1162, 'total_tokens': 1325, 'completion_time': 0.128667705, 'prompt_time': 0.175946418, 'queue_time': None, 'total_time': 0.304614123}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2245759c-5a61-49f7-8863-914904642a11-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_s3wr', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_5hjy', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1162, 'output_tokens': 163, 'total_tokens': 1325}),
 ToolMessage(